# ライブラリのインポート

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import math
import warnings
warnings.simplefilter('ignore')

# 画像の読み込み

In [ ]:
from cv2 import cvtColor
img = cv2.imread("./cell.jpg")
# plt.imshow(cvtColor(img, cv2.COLOR_RGB2BGR))
original = img.copy()

# 赤色を抽出

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# 赤色のHSVの値域1
hsv_min = np.array([0, 64, 0])
hsv_max = np.array([30, 255, 255])
mask1 = cv2.inRange(hsv, hsv_min, hsv_max)

# 赤色のHSVの値域2
hsv_min = np.array([150, 64, 0])
hsv_max = np.array([179, 255, 255])
mask2 = cv2.inRange(hsv, hsv_min, hsv_max)

# 赤色領域のマスク（255：赤色、0：赤色以外）
mask = mask1 + mask2

# マスキング処理
img_red = cv2.bitwise_and(img, img, mask=mask)

# plt.imshow(cvtColor(img_red, cv2.COLOR_RGB2BGR))

# グレースケールに

In [ ]:
img_gray = cv2.cvtColor(img_red, cv2.COLOR_BGR2GRAY)
# plt.imshow(cvtColor(img_gray, cv2.COLOR_RGB2BGR))

# 二値化画像の生成

In [ ]:
th, img_thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# plt.imshow(cvtColor(img_thresh, cv2.COLOR_RGB2BGR))

# モルフォジー処理

In [ ]:
#短形カーネルの生成
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
opening = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel, iterations=1)
close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)

# 細胞のカウント

In [ ]:
cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [ ]:
minimum_area = 20
average_cell_area = 500
connected_cell_area = 1000
cells = 0
for c in cnts:
    area = cv2.contourArea(c)
    if area > minimum_area:
        cv2.drawContours(original, [c], -1, (36,255,12), 2)
        if area > connected_cell_area:
            cells += math.ceil(area / average_cell_area)
        else:
            cells += 1
print('Cells: {}'.format(cells))